지난 노드에서 __챗봇과 번역기는 같은 집안__이라고 했던 말을 기억하시나요?
앞서 배운 Seq2seq번역기와 Transfomer번역기에 적용할 수도 있겠지만, 이번 노드에서 배운 번역기 성능 측정법을 챗봇에도 적용해봅시다. 배운 지식을 다양하게 활용할 수 있는 것도 중요한 능력이겠죠. 이번 프로젝트를 통해서 챗봇과 번역기가 같은 집안인지 확인해보세요!

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.4.1


---

# Step 1. 데이터 다운로드

아래 링크에서 ChatbotData.csv 를 다운로드해 챗봇 훈련 데이터를 확보합니다. csv 파일을 읽는 데에는 pandas 라이브러리가 적합합니다. 읽어 온 데이터의 질문과 답변을 각각 questions, answers 변수에 나눠서 저장하세요!

- [songys/Chatbot_data](songys/Chatbot_data)

## 데이터 로드 & 확인

In [23]:
# data load
path_to_file = os.getenv('HOME')+'/aiffel/songys/Chatbot_data/ChatbotData.csv'
data = pd.read_csv(path_to_file)

In [24]:
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


## 데이터의 질문과 답변 분할 저장

In [25]:
src = []
tgt = []
for s,t in zip(data['Q'],data['A']):
    src.append(str(s))
    tgt.append(str(t))

for s,t in zip(src[:20],tgt[:20]):
    print(">> ", s, "+",t)

>>  12시 땡! + 하루가 또 가네요.
>>  1지망 학교 떨어졌어 + 위로해 드립니다.
>>  3박4일 놀러가고 싶다 + 여행은 언제나 좋죠.
>>  3박4일 정도 놀러가고 싶다 + 여행은 언제나 좋죠.
>>  PPL 심하네 + 눈살이 찌푸려지죠.
>>  SD카드 망가졌어 + 다시 새로 사는 게 마음 편해요.
>>  SD카드 안돼 + 다시 새로 사는 게 마음 편해요.
>>  SNS 맞팔 왜 안하지ㅠㅠ + 잘 모르고 있을 수도 있어요.
>>  SNS 시간낭비인 거 아는데 매일 하는 중 + 시간을 정하고 해보세요.
>>  SNS 시간낭비인데 자꾸 보게됨 + 시간을 정하고 해보세요.
>>  SNS보면 나만 빼고 다 행복해보여 + 자랑하는 자리니까요.
>>  가끔 궁금해 + 그 사람도 그럴 거예요.
>>  가끔 뭐하는지 궁금해 + 그 사람도 그럴 거예요.
>>  가끔은 혼자인게 좋다 + 혼자를 즐기세요.
>>  가난한 자의 설움 + 돈은 다시 들어올 거예요.
>>  가만 있어도 땀난다 + 땀을 식혀주세요.
>>  가상화폐 쫄딱 망함 + 어서 잊고 새출발 하세요.
>>  가스불 켜고 나갔어 + 빨리 집에 돌아가서 끄고 나오세요.
>>  가스불 켜놓고 나온거 같아 + 빨리 집에 돌아가서 끄고 나오세요.
>>  가스비 너무 많이 나왔다. + 다음 달에는 더 절약해봐요.


---

# Step 2. 데이터 정제

아래 조건을 만족하는 preprocess_sentence() 함수를 구현하세요.

1. 영문자의 경우, __모두 소문자로 변환__합니다.
2. 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 __정규식을 활용하여 모두 제거__합니다.

_문장부호 양옆에 공백을 추가하는 등 이전과 다르게 생략된 기능들은 우리가 사용할 토크나이저가 지원하기 때문에 굳이 구현하지 않아도 괜찮습니다!_

In [26]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^0-9ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()
    
    return sentence

---

# Step 3. 데이터 토큰화

토큰화에는 _KoNLPy_의 mecab 클래스를 사용합니다.

아래 조건을 만족하는 build_corpus() 함수를 구현하세요!

1. __소스 문장 데이터__와 __타겟 문장 데이터__를 입력으로 받습니다.
2. 데이터를 앞서 정의한 preprocess_sentence() 함수로 __정제하고, 토큰화__합니다.
3. 토큰화는 __전달받은 토크나이즈 함수를 사용__합니다. 이번엔 mecab.morphs 함수를 전달하시면 됩니다.
4. 토큰의 개수가 일정 길이 이상인 문장은 __데이터에서 제외__합니다.
5. __중복되는 문장은 데이터에서 제외__합니다. 소스 : 타겟 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!

구현한 함수를 활용하여 questions 와 answers 를 각각 que_corpus , ans_corpus 에 토큰화하여 저장합니다.

In [27]:
from collections import Counter
from konlpy.tag import Mecab

vocab_size = 20000
tokenizer = Mecab()
max_len = 50

def build_corpus(src, tgt, l, num_words=vocab_size, dup=0):
    if dup == 0:
        sen_idx = {}
        src_u = []
        tgt_u = []

        for sen1, sen2 in zip(src, tgt):
            if sen1 not in sen_idx:
                sen_idx[sen1] = 1
                src_u.append(sen1)
                tgt_u.append(sen2)

        sen_idx = {}
        src = []
        tgt = []

        for sen1, sen2 in zip(src_u, tgt_u):
            if sen2 not in sen_idx:
                sen_idx[sen2] = 1
                src.append(sen1)
                tgt.append(sen2)

    src_p = []
    tgt_p = []
    
    for s, t in zip(src, tgt):
        src_p.append(preprocess_sentence(s))
        tgt_p.append(preprocess_sentence(t))

    src_tok = []
    tgt_tok = []
    word_tok = []

    for s in src_p:
        tmp = tokenizer.morphs(s)
        src_tok.append(tmp)
        word_tok.append(tmp)

    for t in tgt_p:
        tmp = tokenizer.morphs(t)
        tgt_tok.append(tmp)
        word_tok.append(tmp)

    words = np.concatenate(word_tok).tolist()
    counter = Counter(words)
    counter = counter.most_common(num_words-4)
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<EOS>'] + [key for key, _ in counter]
    
    # 사전 구성
    word_to_index = {word: index for index, word in enumerate(vocab)}

    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]
    # 변환 text to index
    src_data = list(map(wordlist_to_indexlist, src_tok))
    tgt_data = list(map(wordlist_to_indexlist, tgt_tok))

    src_l = []
    tgt_l = []

    for s, t in zip(src_data, tgt_data):
        if len(s) <= l and len(t) <= l:
            src_l.append(s)
            tgt_l.append(t)

    return src_l, tgt_l, word_to_index


que_corpus, ans_corpus, word_to_index = build_corpus(src, tgt, max_len)
for q, a in zip(que_corpus[:20], ans_corpus[:20]):
    print(q, a)

[2057, 209, 2581, 105] [264, 9, 136, 9, 39, 4]
[284, 3559, 599, 1043, 13] [624, 17, 1492, 4]
[279, 2058, 480, 65, 271, 261, 9, 11, 44, 35] [275, 16, 672, 10, 34, 4]
[3560, 1126, 36] [5180, 5, 5181, 19, 34, 4]
[3561, 1535, 3562, 13] [121, 2000, 188, 6, 24, 51, 5182, 4]
[544, 190, 1044, 132, 42, 7, 19, 627] [57, 116, 11, 15, 8, 38, 23, 15, 30, 4]
[544, 73, 1536, 102, 14, 21, 46, 412, 7, 6, 146] [73, 8, 2452, 11, 17, 18, 12, 4]
[544, 18, 37, 22, 66, 1127, 11, 35, 152, 17, 521] [2864, 7, 6, 846, 254, 4]
[494, 318, 17] [90, 27, 23, 217, 14, 28, 4]
[494, 16, 180, 102, 24, 10, 35] [180, 31, 969, 12, 4]
[3563, 33, 206, 40, 3564] [266, 16, 121, 3278, 14, 28, 4]
[3565, 15, 239, 2059, 1757] [2059, 8, 3279, 56, 12, 4]
[3566, 3567, 3568, 2582] [291, 123, 11, 513, 1162, 7, 12, 4]
[2060, 1045, 1758, 11, 1537, 13] [523, 228, 26, 844, 143, 982, 11, 562, 12, 4]
[2060, 325, 54, 78, 825, 35, 4] [424, 140, 26, 6, 67, 1873, 17, 49, 4]
[2060, 325, 3569, 863, 1128, 25, 13] [608, 7, 24, 707, 105]
[454, 436, 33

---

# Step 4. Augmentation

우리에게 주어진 데이터는 __1만 개가량으로 적은 편__에 속합니다. 이럴 때에 사용할 수 있는 테크닉을 배웠으니 활용해봐야겠죠? __Lexical Substitution을 실제로 적용__해보도록 하겠습니다.

아래 링크를 참고하여 __한국어로 사전 훈련된 Embedding 모델을 다운로드__합니다. Korean (w) 가 Word2Vec으로 학습한 모델이며 용량도 적당하므로 사이트에서 Korean (w)를 찾아 다운로드하고, ko.bin 파일을 얻으세요!

- [Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors)

다운로드한 모델을 활용해 데이터를 __Augmentation__ 하세요! 앞서 정의한 lexical_sub() 함수를 참고하면 도움이 많이 될 겁니다.

_Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록, 이후엔 반대로 원본 que_corpus 와 Augmentation된 ans_corpus 가 병렬을 이루도록 하여 __전체 데이터가 원래의 3배가량으로 늘어나도록__ 합니다._

In [34]:
import gensim

wv = gensim.models.Word2Vec.load('~/aiffel/songys/ko/ko.bin')

In [35]:
def lexical_sub(sentence, word2vec, top=0):
    import random

    res = ""
    toks = sentence.split()
    
    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[top][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

In [ ]:
from tqdm import tqdm_notebook

src_corpus = []
tgt_corpus = []
for idx in tqdm_notebook(range(7731)):
    
    old_src = []
    for w in que_corpus[idx]:
        ow = index_to_word[w]
        old_src.append(ow)
    old_src = ' '.join(old_src)

    old_tgt = []
    for w in ans_corpus[idx]:
        ow = index_to_word[w]
        old_tgt.append(ow)
    old_tgt = ' '.join(old_tgt)
    
    new_src = [None]*3
    new_tgt = [None]*3
    
    new_src[0] = old_src
    new_tgt[0] = old_tgt
    
    new_src[1] = lexical_sub(old_src, wv)
    new_src[2] = lexical_sub(old_src, wv, 1)
    
    new_tgt[1] = lexical_sub(old_tgt, wv)
    new_tgt[2] = lexical_sub(old_tgt, wv, 1)  
    
    for i in new_src:
        for j in new_tgt:
            if i is not None and j is not None:
                src_corpus.append(i)
                tgt_corpus.append(j)
    
    
print(src_corpus[:20])
print(src[:20])
print(len(src_corpus))

---

# Step 5. 데이터 벡터화

타겟 데이터인 ans_corpus 에 \<start> 토큰과 \<end> 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행합니다. 우리가 구축한 ans_corpus 는 list 형태이기 때문에 아주 쉽게 이를 해결할 수 있답니다!

``` python
sample_data = ["12", "시", "땡", "!"]

print(["<start>"] + sample_data + ["<end>"])
```

1. 위 소스를 참고하여 타겟 데이터 전체에 \<start> 토큰과 \<end> 토큰을 추가해 주세요!

챗봇 훈련 데이터의 가장 큰 특징 중 하나라고 하자면 바로 __소스 데이터와 타겟 데이터가 같은 언어를 사용한다는 것__이겠죠. 앞서 배운 것처럼 이는 Embedding 층을 공유했을 때 많은 이점을 얻을 수 있습니다.

2. 특수 토큰을 더함으로써 ans_corpus 또한 완성이 되었으니, que_corpus 와 결합하여 __전체 데이터에 대한 단어 사전을 구축__하고 __벡터화하여 enc_train 과 dec_train__ 을 얻으세요!

In [37]:
new_que_corpus, new_ans_corpus, word_to_index = build_corpus(src_corpus,tgt_corpus,max_len,dup=1)

In [39]:
print(len(new_que_corpus))
print(len(new_ans_corpus))

58214
58214


In [40]:
index_to_word = {index:word for word, index in word_to_index.items()}

In [41]:
ans = []
for a in new_ans_corpus:
    ans.append([word_to_index["<BOS>"]] + a + [word_to_index["<EOS>"]])

In [42]:
enc_tensor = tf.keras.preprocessing.sequence.pad_sequences(new_que_corpus, padding='post')
dec_tensor = tf.keras.preprocessing.sequence.pad_sequences(ans, padding='post')

enc_train, enc_val, dec_train, dec_val = \
train_test_split(enc_tensor, dec_tensor, test_size=0.01)

print("enc_train :", len(enc_train), "enc_val :", len(enc_val))
print("dec_train :", len(dec_train), "dec_val :",len(dec_val))

enc_train : 57631 enc_val : 583
dec_train : 57631 dec_val : 583


In [43]:
print(len(enc_train[0]))
print(len(dec_train[0]))

33
43


---

# Step 6. 훈련하기

앞서 번역 모델을 훈련하며 정의한 Transformer 를 그대로 사용하시면 됩니다! 대신 데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합을 피할 수 있습니다. 모델을 훈련하고 아래 예문에 대한 답변을 생성하세요! __가장 멋진 답변__과 __모델의 하이퍼파라미터__를 제출하시면 됩니다.

``` python
# 예문
1. 지루하다, 놀러가고 싶어.
2. 오늘 일찍 일어났더니 피곤하다.
3. 간만에 여자친구랑 데이트 하기로 했어.
4. 집에 있는다는 소리야.

---

# 제출

Translations
> 1. 잠깐 쉬 어도 돼요 . <end>
> 2. 맛난 거 드세요 . <end>
> 3. 떨리 겠 죠 . <end>
> 4. 좋 아 하 면 그럴 수 있 어요 . <end>

Hyperparameters
> n_layers: 1
> d_model: 368
> n_heads: 8
> d_ff: 1024
> dropout: 0.2

Training Parameters
> Warmup Steps: 1000
> Batch Size: 64
> Epoch At: 10
```

In [44]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table
print("슝=3")

슝=3


In [46]:
# Mask  생성하기
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask
print("슝=3")

슝=3


In [47]:
# Multi Head Attention 구현
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads

        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions


    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x


    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights
print("슝=3")

슝=3


In [48]:
# Position-wise Feed Forward Network 구현
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)

        return out
print("슝=3")

슝=3


In [49]:
# Encoder의 레이어 구현
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, enc_attn
print("슝=3")

슝=3


In [50]:
# Decoder 레이어 구현
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn
print("슝=3")

슝=3


In [51]:

# Encoder 구현
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns
print("슝=3")

슝=3


In [52]:

# Decoder 구현
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]


    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns
print("슝=3")

슝=3


In [53]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out


    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

print("슝=3")

슝=3


In [54]:
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성
transformer = Transformer(
    n_layers=2,
    d_model=128,
    n_heads=8,
    d_ff=256,
    src_vocab_size=vocab_size,
    tgt_vocab_size=vocab_size,
    pos_len=42,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

d_model = 128
print("슝=3")

슝=3


In [55]:
# Learning Rate Scheduler 구현
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()

        self.d_model = d_model
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)

        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

print("슝=3")

슝=3


In [56]:
# Learning Rate 인스턴스 선언 & Optimizer 구현
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)
print("슝=3")

슝=3


In [57]:
# Loss Function 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

print("슝=3")

슝=3


In [58]:
# Train Step 정의
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

print("슝=3")

슝=3


In [59]:
# 훈련시키기
from tqdm import tqdm_notebook 

BATCH_SIZE = 64
EPOCHS = 5

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

/home/aiffel-dj24/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/901 [00:00<?, ?it/s]

  0%|          | 0/901 [00:00<?, ?it/s]

  0%|          | 0/901 [00:00<?, ?it/s]

  0%|          | 0/901 [00:00<?, ?it/s]

  0%|          | 0/901 [00:00<?, ?it/s]

---

# Step 7. 성능 측정하기

챗봇의 경우, 올바른 대답을 하는지가 중요한 평가지표입니다. 올바른 답변을 하는지 눈으로 확인할 수 있겠지만, 많은 데이터의 경우는 모든 결과를 확인할 수 없을 것입니다. 주어잔 질문에 적절한 답변을 하는지 확인하고, BLEU Score를 계산하는 calculate_bleu() 함수도 적용해보세요

In [60]:
# !pip install nltk # nltk가 설치되어 있지 않은 경우 주석 해제
from nltk.translate.bleu_score import sentence_bleu

reference = "많 은 자연어 처리 연구자 들 이 트랜스포머 를 선호 한다".split()
candidate = "적 은 자연어 학 개발자 들 가 트랜스포머 을 선호 한다 요".split()

print("원문:", reference)
print("번역문:", candidate)
print("BLEU Score:", sentence_bleu([reference], candidate))

원문: ['많', '은', '자연어', '처리', '연구자', '들', '이', '트랜스포머', '를', '선호', '한다']
번역문: ['적', '은', '자연어', '학', '개발자', '들', '가', '트랜스포머', '을', '선호', '한다', '요']
BLEU Score: 8.190757052088229e-155


/home/aiffel-dj24/anaconda3/envs/aiffel/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/aiffel-dj24/anaconda3/envs/aiffel/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [61]:
print("1-gram:", sentence_bleu([reference], candidate, weights=[1, 0, 0, 0]))
print("2-gram:", sentence_bleu([reference], candidate, weights=[0, 1, 0, 0]))
print("3-gram:", sentence_bleu([reference], candidate, weights=[0, 0, 1, 0]))
print("4-gram:", sentence_bleu([reference], candidate, weights=[0, 0, 0, 1]))

1-gram: 0.5
2-gram: 0.18181818181818182
3-gram: 2.2250738585072626e-308
4-gram: 2.2250738585072626e-308


/home/aiffel-dj24/anaconda3/envs/aiffel/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/aiffel-dj24/anaconda3/envs/aiffel/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [62]:
from nltk.translate.bleu_score import SmoothingFunction

def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                         candidate,
                         weights=weights,
                         smoothing_function=SmoothingFunction().method1)  # smoothing_function 적용

print("BLEU-1:", calculate_bleu(reference, candidate, weights=[1, 0, 0, 0]))
print("BLEU-2:", calculate_bleu(reference, candidate, weights=[0, 1, 0, 0]))
print("BLEU-3:", calculate_bleu(reference, candidate, weights=[0, 0, 1, 0]))
print("BLEU-4:", calculate_bleu(reference, candidate, weights=[0, 0, 0, 1]))

print("\nBLEU-Total:", calculate_bleu(reference, candidate))

BLEU-1: 0.5
BLEU-2: 0.18181818181818182
BLEU-3: 0.010000000000000004
BLEU-4: 0.011111111111111112

BLEU-Total: 0.05637560315259291


In [63]:
# translate()

def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)
    
    print(sentence)
    s = src_tokenizer.morphs(sentence)
    tokens = []
    for i in s:
        tokens.append(word_to_index[i])
    pieces = tokens        
        
    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([1], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if predicted_id == 3:
            #result = tgt_tokenizer.decode_ids(ids)
            result = []
            for i in ids:
                result.append(index_to_word[i])
            print(result)
            
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    #result = tgt_tokenizer.decode_ids(ids)
    result = ''
    for i in ids:
        result += index_to_word[i]
    
    return pieces, result, enc_attns, dec_attns, dec_enc_attns

def translate(sentence, model, src_tokenizer, tgt_tokenizer):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model, src_tokenizer, tgt_tokenizer)

    return result
print("슝=3")

슝=3


In [65]:
def eval_bleu(src_corpus, tgt_corpus, verbose=True):
    total_score = 0.0
    sample_size = len(tgt_corpus)

    for idx in tqdm_notebook(range(sample_size)):
        src_tokens = src_corpus[idx]
        tgt_tokens = tgt_corpus[idx]

        src_sentence = []
        tgt_sentence = []

        for w in src_tokens:
            if w != 0:
                ow = index_to_word[w]
                src_sentence.append(ow)

        src_sentence = ' '.join(src_sentence)

        for w in tgt_tokens:
            if w != 0 and w != 1 and w != 3:
                ow = index_to_word[w]
                tgt_sentence.append(ow)

        tgt_sentence = ' '.join(tgt_sentence)

        reference = preprocess_sentence(tgt_sentence).split()
        candidate = translate(src_sentence, transformer, tokenizer, tokenizer)

        score = sentence_bleu([reference], candidate,
                              smoothing_function=SmoothingFunction().method1)
        total_score += score

        if verbose:
            print("Source Sentence: ", src_sentence)
            print("Model Prediction: ", candidate)
            print("Real: ", reference)
            print("Score: %lf\n" % score)

    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)


print("슝=3")

슝=3


In [66]:
eval_bleu(enc_val[:3], dec_val[:3], True)

/home/aiffel-dj24/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/3 [00:00<?, ?it/s]

싸운다 의 기준 그러 뭘까 ?
['감정', '이', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '지요', '.']
Source Sentence:  싸운다 의 기준 그러 뭘까 ?
Model Prediction:  ['감정', '이', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '지요', '.']
Real:  ['감정', '이', '상', '하', '면', '싸우', '는', '거', '죠', '.']
Score: 0.021239

아빠 놀 드 담배 그만 피웠으면
['금연', '프로그램', '을', '소개', '해', '보', '세요', '.']
Source Sentence:  아빠 놀 드 담배 그만 피웠으면
Model Prediction:  ['금연', '프로그램', '을', '소개', '해', '보', '세요', '.']
Real:  ['금연', '프로그램', '을', '소개', '해', '보', '세요', '.']
Score: 1.000000

관심 있 는 여자 애 에게 의미 있 는 선물 을 주 고 싶 은데 뭘 주 면 좋아할까요 ?
['필요', '한', '게', '뭔지', '보', '세요', '.']
Source Sentence:  관심 있 는 여자 애 에게 의미 있 는 선물 을 주 고 싶 은데 뭘 주 면 좋아할까요 ?
Model Prediction:  ['필요', '한', '게', '뭔지', '보', '세요', '.']
Real:  ['필요', '한', '게', '뭔지', '살펴보', '세요', '.']
Score: 0.488923

Num of Sample: 3
Total S

In [68]:
eval_bleu(enc_val[::10], dec_val[::10], verbose=False)

/home/aiffel-dj24/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/59 [00:00<?, ?it/s]

싸운다 의 기준 그러 뭘까 ?
['감정', '이', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '면', '싸우', '지요', '.']
술 먹 고 카드 긁 었 어
['술', '이', '웬수', '예요', '.']
친한 연인 의 구 여친 이랑 사귀 어도 되 나 .
['친구', '라면', '많이', '실망', '할', '거', '라', '생각', '해요', '.']
유명 인 되 고 싶 어
['저', '도', '괜찮', '아', '오', '세요', '.']
여자 친구 가 변해 가 는 듯 같 아 .
['모습', '간혹', '보', '는', '건', '어떨까', '요', '.']
짝 이듬해 관련 돼서 악몽 을 자주 꾸 는데 불안 함 .
['악몽', '되', '긴', '하', '겠', '네요', '.']
눈 그러 계속 뻑뻑 해
['인공', '눈물', '눈물', '인', '넣', '어', '졌', '죠', '.']
사랑 하 는 사람 이랑 헤어져야 해
['정말', '바라', '면', '돼요', '.']
짝사랑 시작 하 는 데 감정 기복 이 심해 .
['짝사랑', '이', '그런', '건가', '봐요', '.']
이 별후 동침 그리고
['달라진', '게', '많', '았', '겠', '네요', '.']
이 길 이 니 에게 맞 는 걸까
['잘', '가', '고', '있', '었', '을', '거', '예요', '.']
어떤 남자 애 가 내일 . . 하 냐는데 나 한테 관심 있 나 ?
['시간', '하', '고', '있', '는지', '찾아보', '세요', '.']
일 하 는 이곳 근처 가 서 염탐 하 고 왔 네 .
['그분', '에', '이유', '가', '필요', '한가요', '.']
몸값 다 없 어 졌 어
['돈', '있', '는', '있', '어요', '.']
휴양지 가 는데 싶 다
['저', '도', 

InvalidArgumentError: Incompatible shapes: [1,43,128] vs. [1,42,128] [Op:AddV2]

In [69]:
translate("지루하다, 놀러가고 싶어.", transformer, tokenizer, tokenizer)

지루하다 , 놀러가고 싶어 .
['힘내', '지', '않', '을까', '하', '면', '후', '에', '게', '좋', '을', '거', '예요', '.']


['힘내', '지', '않', '을까', '하', '면', '후', '에', '게', '좋', '을', '거', '예요', '.']

In [70]:
translate("오늘 일찍 일어났더니 피곤하다.", transformer, tokenizer, tokenizer)

오늘 일찍 일어났더니 피곤하다 .
['피', '작', '은', '후련', '하', '네요', '.']


['피', '작', '은', '후련', '하', '네요', '.']

In [71]:
translate("간만에 여자친구랑 데이트 하기로 했어.", transformer, tokenizer, tokenizer)

간만에 여자친구랑 데이트 하기로 했어 .
['득템', '하', '길', '바라', '요', '.']


['득템', '하', '길', '바라', '요', '.']

In [72]:
translate("집에 있는다는 소리야.", transformer, tokenizer, tokenizer)

집에 있는다는 소리야 .
['어떻게', '하', '면서', '면', '돼요', '.']


['어떻게', '하', '면서', '면', '돼요', '.']

---

__1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가?__  

- 챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.

__2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가?__  

- 과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시되었다.

__3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가?__  

- 주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.

---

# 회고